In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)


def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if (len(img.shape) > 2):
        channel_count = img.shape[2]
        ignore_mask_color = (255,)*channel_count
    else:
        ignore_mask_color = 255
    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

In [4]:
def get_lines(lines, color=[255, 0, 0]):
    right_lines = []
    left_lines = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        y = y2 - y1
        x = x2 - x1
        slope = y / x
        if slope < -0.3:
            right_lines.append(line)
        elif slope > 0.3:
            left_lines.append(line)
    return right_lines, left_lines

In [5]:
def get_points(lines):
    points = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        points.append((x1, y1))
        points.append((x2, y2))
    points = np.array(points)
    return points

In [6]:
def fitted_lines(right_points, left_points, color=[255, 0, 0], thickness=9):
    right_line = cv2.fitLine(right_points, cv2.DIST_L2, 0, 0.01, 0.01)
    left_line = cv2.fitLine(left_points, cv2.DIST_L2, 0, 0.01, 0.01)

    return right_line, left_line

In [7]:
left_line_ys = []
left_line_xs = []
right_slopes = []
left_slopes = []
right_line_ys = []
right_line_xs = []

In [8]:
def listed_values(right_line, left_line):
    right_m = right_line[1] / right_line[0]
    left_m = left_line[1] / left_line[0]
    
    right_slopes.append(right_m)
    left_slopes.append(left_m)
    right_line_ys.append(right_line[3])
    right_line_xs.append(right_line[2])    
    left_line_ys.append(left_line[3])
    left_line_xs.append(left_line[2])
    return right_slopes, left_slopes, right_line_ys, right_line_xs, left_line_ys, left_line_xs

In [9]:
def compute_means(right_slopes, left_slopes, right_line_ys, right_line_xs, left_line_ys, left_line_xs):
    right_line_y_avg = np.mean(right_line_ys, keepdims=True)
    right_line_x_avg = np.mean(right_line_xs, keepdims=True)
    
    left_line_y_avg = np.mean(left_line_ys, keepdims=True)
    left_line_x_avg = np.mean(left_line_xs, keepdims=True)
    right_m_avg = np.mean(right_slopes)
    left_m_avg = np.mean(left_slopes)
    return right_m_avg, right_line_x_avg, right_line_y_avg, left_m_avg, left_line_x_avg, left_line_y_avg

In [10]:
def draw_lines(img, right_m_avg, right_line_x_avg, right_line_y_avg, left_m_avg, left_line_x_avg, left_line_y_avg, color=[255, 0,0], thickness=5):
    y_ini = img.shape[0]
    y_fin = img.shape[0] / 1.6

    right_ini_x = ((y_ini - right_line_y_avg) / right_m_avg) + right_line_x_avg
    right_fin_x = ((y_fin - right_line_y_avg) / right_m_avg) + right_line_x_avg
      
    left_ini_x = ((y_ini - left_line_y_avg) / left_m_avg) + left_line_x_avg
    left_fin_x = ((y_fin - left_line_y_avg) / left_m_avg) + left_line_x_avg
    
    cv2.line(img, (right_ini_x, y_ini), (right_fin_x, int(y_fin)), color, thickness)
    cv2.line(img, (left_ini_x, y_ini), (left_fin_x, int(y_fin)), color, thickness)

In [11]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    right_lines, left_lines = get_lines(lines)
    right_points = get_points(right_lines)
    left_points = get_points(left_lines)
    rfitted_line, lfitted_line = fitted_lines(right_points, left_points)
    right_slopes, left_slopes, right_line_ys, right_line_xs, left_line_ys, left_line_xs = listed_values(rfitted_line, lfitted_line)
    right_m_avg, right_line_x_avg, right_line_y_avg, left_m_avg, left_line_x_avg, left_line_y_avg = compute_means(right_slopes, left_slopes, right_line_ys, right_line_xs, left_line_ys, left_line_xs)

    draw_lines(line_img, right_m_avg, right_line_x_avg, right_line_y_avg, left_m_avg, left_line_x_avg, left_line_y_avg)
    return line_img


def weighted_img(initial_img, img, alpha=0.8, beta=1.0, gamma = 0.):
    return cv2.addWeighted(initial_img, alpha, img,  beta, gamma)

In [12]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [13]:
def process_image(initial_img):

    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    img = grayscale(initial_img)
    img = canny(img, 50, 150)
    img = gaussian_blur(img, 5)
    vertices = np.array([[(0, img.shape[0]), 
                      (470, 300),
                      (490, 300),
                      (img.shape[1] + 80, img.shape[0])]])
    img = region_of_interest(img, vertices)
    img = hough_lines(img, 2, np.pi / 180, 50, 10, 250)
    img = weighted_img(initial_img, img)
    return img

In [14]:
video_capture = cv2.VideoCapture('../test_videos/solidYellowLeft.mp4')
while (video_capture.isOpened()):
    ret, frame = video_capture.read()
    if ret:
        
        output = process_image(frame)
        cv2.imshow('frame',output)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
# Release everything if job is finished
video_capture.release()
cv2.destroyAllWindows()

In [55]:
white_output = '../test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("../test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   4%|▍         | 9/221 [00:00<00:02, 82.83it/s, now=None]

Moviepy - Building video ../test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video ../test_videos_output/solidWhiteRight.mp4



Moviepy - Done !
Moviepy - video ready ../test_videos_output/solidWhiteRight.mp4
CPU times: user 4.25 s, sys: 75.6 ms, total: 4.33 s
Wall time: 3.45 s


In [56]:
yellow_output = '../test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('../test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

t:   1%|▏         | 9/681 [00:00<00:08, 81.54it/s, now=None]

Moviepy - Building video ../test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video ../test_videos_output/solidYellowLeft.mp4



Moviepy - Done !
Moviepy - video ready ../test_videos_output/solidYellowLeft.mp4
CPU times: user 14.2 s, sys: 303 ms, total: 14.5 s
Wall time: 11 s
